In [1]:
import sys
sys.version

'3.6.12 |Anaconda, Inc.| (default, Sep  8 2020, 23:10:56) \n[GCC 7.3.0]'

# spark config

In [2]:
import pyspark
from pyspark import SparkContext, SparkConf

conf = SparkConf().setMaster("local").setAppName("page_rank")
sc = SparkContext(conf=conf).getOrCreate()

# hyper parameter setting

In [3]:
beta = 0.8
node = 10876
iterator = 20

# read data

In [11]:
with open('p2p-Gnutella04.txt') as fp:
    data = fp.readlines()


data = data[4:] # filter unnecessary info
for i in range(len(data)):
    data[i] = data[i].strip().split()# Q: 要改\t
    data[i][0] = int(data[i][0])
    data[i][1] = int(data[i][1])
# data

# 得到M的weight
## from_to 代表graph的連線狀況，(s[0],s[1])分別代表from及to
## out_degree用來計算每個node的outdegree數目
## transistion分別代表(from,(to,out_degree))

In [12]:
from_to = sc.parallelize(data).map(lambda s:(s[0], s[1]))
out_degree = from_to.groupByKey().map(lambda s:(s[0],1/len(s[1])))
transition = from_to.join(out_degree)

# 用state代表目前的狀態
## 初始化得值為1/N, N是node數量

In [13]:
state = [(i, 1/node) for i in range(node)]
state = sc.parallelize(state)

In [14]:
# transition.collect()

[(0, (1, 0.5)),
 (0, (2, 0.5)),
 (2, (0, 0.3333333333333333)),
 (2, (3, 0.3333333333333333)),
 (2, (4, 0.3333333333333333)),
 (4, (0, 0.5)),
 (4, (3, 0.5)),
 (1, (3, 1.0))]

# main function
## state.join(transition).map()用來計算rj的值
## 計算過後的j會當成key,rj是value,再用reduceByKey得到sum(rj)
## state.value.sum()得到state的總和，可以用來作normalize
## 最後在進行renormalize即可得到結果

In [8]:
for i in range(iterator):
    state = state.join(transition).map(lambda s:(s[1][1][0],beta*s[1][0]*s[1][1][1])).reduceByKey(lambda x, y:x+y)
    sum_of_val = state.values().sum()
    # renormalize
    state = state.mapValues(lambda x : x + (1-sum_of_val)/node)
    state

# 依據rank的value排序，並且得到前10高的value

In [9]:
result = state.top(10 ,lambda s:s[1])

# 將結果寫回檔案

In [10]:
with open('Outputfile.txt', 'w') as fp:
    for i in range(len(result)):
        fp.write(str(result[i][0])+'\t'+str(result[i][1])+'\n')